In [28]:
#Loading in required packages
import pandas as pd
import numpy as np
import sys
import pyemd
import inspect
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 

#specifying local system path for remaining functions files
sys.path.insert(1, '/Users/Cylita/Desktop/insight-ds-project_news-link/scripts')

#self written functions and scripts that will be needed
import text_normalization_funs as TN
import similiarity_funcs as SIM

#Importing functions
import gensim 
from gensim.models import Word2Vec 

import spacy
nlp = spacy.load("en_core_web_lg")

import wmd
nlp2 = spacy.load('en_core_web_lg', create_pipeline=wmd.WMD.create_spacy_pipeline)
nlp2.add_pipe(wmd.WMD.SpacySimilarityHook(nlp2), last=True)


In [2]:
#Loading in data 
#Loading in the fully cleaned corpus and validation metrics
sys.path.insert(1, '/Users/Cylita/Desktop/insight-ds-project_news-link/data/cleaned/')
news_corpus = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/cleaned/Final_Cleaned_Corpus.txt')

In [4]:
#Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/Cylita/Desktop/insight-ds-project_news-link/scripts/GoogleNews-vectors-negative300.bin', binary=True)


In [46]:
#Creating a test story:

with open('/Users/Cylita/Desktop/insight-ds-project_news-link/data/cleaned/TestStory_1.txt', 'r') as file:
    TestStory1 = file.read().replace('\n', '')
with open('/Users/Cylita/Desktop/insight-ds-project_news-link/data/cleaned/TestStory_2.txt', 'r') as file:
    TestStory2 = file.read().replace('\n', '')
with open('/Users/Cylita/Desktop/insight-ds-project_news-link/data/cleaned/TestStory_3.txt', 'r') as file:
    TestStory3 = file.read().replace('\n', '')


In [59]:
#Cleaning the comptext to get it ready fo TF-IDF
cleancomp = TN.normalize_Story(story = TestStory3)

#Computing similarity scores for given dataframe
scores = SIM.SimCorpCreate(corpora = news_corpus['cleaned_text'], directory= '/Users/Cylita/Desktop/insight-ds-project_news-link/scripts/', 
                           querydoc = cleancomp)

#Top Stories
Top_Whole_Stories = SIM.StoryMatch(SimScores = scores, originalDF = news_corpus)


In [48]:
Top_Whole_Stories

,title,mainurl,sim_scores
153,Oil and gas industry must do more to address c...,https://www.cbc.ca/news/business/oil-and-gas-i...,0.123619
1464,'No excitement at all' as oilpatch interest wa...,https://www.cbc.ca/news/canada/calgary/crown-d...,0.121086
1346,The new two solitudes: 'Alberta and the rest o...,https://www.cbc.ca/news/canada/calgary/angus-r...,0.094260
1447,P.E.I. immigration forecast to reach record le...,https://www.cbc.ca/news/canada/prince-edward-i...,0.090523
304,Another record year for SUV sales in Quebec as...,https://www.cbc.ca/news/canada/montreal/quebec...,0.088483
1553,Irving Oil refinery drops 2020 carbon cut pledge,https://www.cbc.ca/news/canada/new-brunswick/i...,0.087952
480,Bank of Canada holds interest rates steady,https://www.cbc.ca/news/business/bank-of-canad...,0.087437
481,Bank of Canada holds interest rates steady,https://www.cbc.cahttps://ici.radio-canada.ca/...,0.087437
288,China's economic growth sinks to lowest level ...,https://www.cbc.ca/news/world/china-economy-we...,0.085211
1264,Alberta leaders cheer court ruling against B.C...,https://www.cbc.ca/news/canada/calgary/trans-m...,0.083661


In [49]:
doc_vector = []

for doc in news_corpus['cleaned_text']:
    testing = nlp(doc)
    doc_vector.append(testing)
    

In [60]:
#Cleaning the comptext to get it ready fo TF-IDF
cleancomp = TN.normalize_Story(story = TestStory3)

In [61]:
query = nlp(cleancomp)

In [62]:
sim_scores = []

for doc in doc_vector:
    score = query.similarity(doc)
    sim_scores.append(score)

In [63]:
tryagain = SIM.StoryMatch_Val (SimScores = sim_scores, originalDF = news_corpus, NumLinks = 10)

In [64]:
tryagain.to_csv("behavor3.csv")


In [35]:
doc_vector2 = []

for doc in news_corpus['cleaned_text']:
    testing = nlp2(doc)
    doc_vector2.append(testing)

In [65]:
query2 = nlp2(cleancomp)

In [66]:
sim_scores2 = []

for doc in doc_vector2:
    score = query2.similarity(doc)
    sim_scores2.append(score)

In [67]:
tryagain2 = SIM.EmbeddingMatch (scores = sim_scores2, originalDF = news_corpus, NumLinks=10)

In [68]:
tryagain2.to_csv("Glove_WMD3.csv")